# Regression Approach

This notebook is dedicated to modelling the problem as a regression problem. Different regression techniques are evaluated, and then the best-performing one is optimized in terms of hyper-parameters

In [ ]:
# Change directory for cleaner path handling
%%capture
%cd ..

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree, linear_model, dummy, kernel_ridge, gaussian_process
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.feature_selection import RFECV

from src.evaluation import compare_models, evaluate_model

In [4]:
df = pd.read_csv("data/processed/tracks.csv")
targets = df['views']
features = [
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
    'duration_ms',
    'popularity'
]

data = df[features]

## Model comparison
Different regression models are compared with their default parameters to establish a baseline. The evaluation includes standard scaling as part of a pipeline, so no further pre-processing is needed.

In [5]:
models = [
    ("Baseline", dummy.DummyRegressor(strategy='mean')),
    ("Linear Regression", linear_model.LinearRegression()),
    ("Polynomial Regression", Pipeline([
        ('poly', PolynomialFeatures(degree=3)),
        ('linear', linear_model.LinearRegression())
    ])),
    ("Decision Tree", tree.DecisionTreeRegressor()),
    ("Kernel Ridge", kernel_ridge.KernelRidge()),
    ("Gaussian Process", gaussian_process.GaussianProcessRegressor()),
]
metrics = [
    'r2',
    'neg_mean_absolute_error',
    'neg_root_mean_squared_error',
]
compare_models(models, metrics, data, targets, regression=True)

Evaluating Baseline
Evaluating Linear Regression
Evaluating Polynomial Regression
Evaluating Decision Tree
Evaluating Kernel Ridge
Evaluating Gaussian Process


,model,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_root_mean_squared_error
0,Baseline,0.006748,0.001785,-0.000490,-6.919707e+06,-1.388126e+07
1,Linear Regression,0.011114,0.002221,0.029208,-6.719111e+06,-1.367420e+07
2,Polynomial Regression,0.516653,0.014402,-2.891988,-7.783316e+06,-2.331822e+07
3,Decision Tree,0.148316,0.003164,-0.749538,-7.938193e+06,-1.835514e+07
4,Kernel Ridge,3.706343,0.060602,-0.090724,-5.855657e+06,-1.449240e+07
5,Gaussian Process,9.938632,1.379045,-2.742904,-1.270916e+07,-2.626599e+07


None of the selected models seem to be able to model the data, resulting in negative R2 scores, and significantly large errors, except linear regression.

### Feature Selection
In an attempt to improve prediction metrics, feature selection is performed via 5-fold cross validation using Recursive Feature Elimination

In [6]:
train_X, test_X, train_y, test_y = train_test_split(data, targets, test_size=.2,random_state=1)
model = Pipeline([
    ('std', StandardScaler()),
    ('reg',linear_model.LinearRegression())
])
selector = RFECV(model, cv=KFold(shuffle=True, random_state=1),scoring="r2", importance_getter='named_steps.reg.coef_')
selector.fit(train_X.values,train_y)
selected_features = np.array(features)[selector.ranking_ == 1]
selected_features

array(['danceability', 'instrumentalness', 'popularity'], dtype='<U16')

In [7]:
compare_models(models, metrics, data[selected_features], targets, regression=True)

Evaluating Baseline
Evaluating Linear Regression
Evaluating Polynomial Regression
Evaluating Decision Tree
Evaluating Kernel Ridge
Evaluating Gaussian Process


,model,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_root_mean_squared_error
0,Baseline,0.003660,0.001998,-0.000490,-6.919707e+06,-1.388126e+07
1,Linear Regression,0.003873,0.001878,0.029636,-6.711314e+06,-1.367122e+07
2,Polynomial Regression,0.009773,0.002390,0.042456,-6.554856e+06,-1.357894e+07
3,Decision Tree,0.028774,0.002477,-0.781480,-7.946711e+06,-1.848165e+07
4,Kernel Ridge,3.998189,0.062590,-0.090376,-5.829469e+06,-1.449011e+07
5,Gaussian Process,9.721217,1.322720,-462.395484,-1.923280e+07,-2.573207e+08


Using the best performing features, polynomial regression seems to achieve higher R2 scores. By further adding L2 regularization, we can further increase the R2 score.

In [60]:
model = Pipeline([
    ('std', StandardScaler()),
    ('poly', PolynomialFeatures(degree=3)),
    ('reg',linear_model.Ridge())
])
params = {
    'reg__alpha' : [0.1,1,10,100,1000,10000],
}
optimizer = GridSearchCV(model, params ,scoring='r2')
optimizer.fit(train_X[selected_features],train_y)

GridSearchCV(estimator=Pipeline(steps=[('std', StandardScaler()),
                                       ('poly', PolynomialFeatures(degree=3)),
                                       ('reg', Ridge())]),
             param_grid={'reg__alpha': [0.1, 1, 10, 100, 1000, 10000]},
             scoring='r2')

In [61]:
optimizer.best_params_

{'reg__alpha': 1000}

In [62]:
optimizer.best_score_

0.043466959443468414

In [63]:
optimizer.score(test_X[selected_features],test_y)

0.03957779938682138